In [ ]:
""" Phase field damage code for Tensile test""" 


from mpi4py import MPI
import numpy as np
from petsc4py import PETSc
import dolfinx
from dolfinx import mesh, fem, io, default_scalar_type
import ufl
import basix
import dolfinx.fem.petsc
from mpi4py import MPI
import dolfinx.io
import dolfinx.mesh
# ----------------------------------------------------
# 1. Mesh Generation & Tagging
# ----------------------------------------------------
# L, H_geom = 1.0, 1.0
# msh = mesh.create_rectangle(
#     comm=MPI.COMM_WORLD, 
#     points=((0.0, -0.5), (1.0, 0.5)), 
#     n=(50, 50), 
#     cell_type=mesh.CellType.triangle
# )
filename = "Model2_xr_0.5_yr_0.0_1x1.msh"  # path to your Gmsh file

mesh_data = dolfinx.io.gmsh.read_from_msh(
    filename, MPI.COMM_WORLD, gdim=2
)
msh = mesh_data.mesh

# Define Boundary Geometries
def top(x):
    return np.isclose(x[1], 0.5)

def bottom(x):
    return np.isclose(x[1], -0.5)

def crack(x):
    return (np.abs(x[1]) < 1e-3) & (x[0] <= 0.2)

# --- CORRECTION: Create MeshTags for Integration (ds) ---
fdim = msh.topology.dim - 1
boundaries = [(1, top), (2, bottom)] # 1=Top, 2=Bottom

facet_indices = []
facet_markers = []

for (marker, locator) in boundaries:
    facets = mesh.locate_entities_boundary(msh, fdim, locator)
    facet_indices.append(facets)
    facet_markers.append(np.full_like(facets, marker))

facet_indices = np.hstack(facet_indices).astype(np.int32)
facet_markers = np.hstack(facet_markers).astype(np.int32)
sorted_facets = np.argsort(facet_indices)
facet_tag = mesh.meshtags(msh, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets])

# Define the measure 'ds' using these tags
ds = ufl.Measure("ds", domain=msh, subdomain_data=facet_tag)
dx = ufl.Measure("dx", domain=msh)
n = ufl.FacetNormal(msh)

# ----------------------------------------------------
# 2. Function Spaces
# ----------------------------------------------------
# P1 Element for Phase Field (Scalar)
el_scalar = basix.ufl.element("Lagrange", msh.topology.cell_name(), 1)
V = fem.functionspace(msh, el_scalar)

# P1 Element for Displacement (Vector)
el_vector = basix.ufl.element("Lagrange", msh.topology.cell_name(), 1, shape=(msh.geometry.dim,))
W = fem.functionspace(msh, el_vector)

# DG0 Space for History Variable
el_dg = basix.ufl.element("DG", msh.topology.cell_name(), 0)
VDG = fem.functionspace(msh, el_dg)

# ----------------------------------------------------
# 3. Boundary Conditions
# ----------------------------------------------------
# Locate DOFs
bdofs = mesh.locate_entities_boundary(msh, fdim, bottom)
tdofs = mesh.locate_entities_boundary(msh, fdim, top)
cdofs = mesh.locate_entities_boundary(msh, fdim, crack)

# BC for Displacement
# Bottom: Fixed y, Free x (or fixed both depending on setup, here fixed both for stability)
bc_bot = fem.dirichletbc(np.array([0, 0], dtype=default_scalar_type), 
                         fem.locate_dofs_topological(W, fdim, bdofs), W)

# Top: Displacement loading on Y
load = fem.Constant(msh, default_scalar_type(0.0))
top_dofs = fem.locate_dofs_topological(W.sub(1), fdim, tdofs)
bc_top = fem.dirichletbc(load, top_dofs, W.sub(1))

bc_u = [bc_bot, bc_top]

# BC for Phase Field (Crack = 1.0)
bc_phi = [] 
# fem.dirichletbc(default_scalar_type(1.0), fem.locate_dofs_topological(V, fdim, cdofs), V)

# ----------------------------------------------------
# 4. Constitutive Relations
# ----------------------------------------------------
Gc = fem.Constant(msh, 2.7)
l = fem.Constant(msh, 0.015)
lmbda = fem.Constant(msh, 121.1538e3)
mu = fem.Constant(msh, 80.7692e3)
k_ell = 1e-6 

def epsilon(u):
    return ufl.sym(ufl.grad(u))

def sigma_0(u):
    return 2.0 * mu * epsilon(u) + lmbda * ufl.tr(epsilon(u)) * ufl.Identity(len(u))

def psi(u):
    return 0.5 * ufl.inner(sigma_0(u), epsilon(u))

# Degraded Stress
def sigma(u, p):
    return ((1 - p)**2 + k_ell) * sigma_0(u)

# --- CORRECTION: Symbolic History Check ---
# This calculates max(psi(u), H_old) symbolically for the variational form
def History_Condition(u_current, H_prev):
    return ufl.conditional(ufl.gt(psi(u_current), H_prev), psi(u_current), H_prev)

# ----------------------------------------------------
# 5. Variational Problems
# ----------------------------------------------------
# Functions
u, v = ufl.TrialFunction(W), ufl.TestFunction(W)
p, q = ufl.TrialFunction(V), ufl.TestFunction(V)

# Actual Solutions
unew = fem.Function(W) # Solution at t_{n+1}
pnew = fem.Function(V) # Solution at t_{n+1}
Hold = fem.Function(VDG) # History at t_{n} (Converged)

# Helper functions for convergence checking
u_iter = fem.Function(W) # For storing previous iteration result
p_iter = fem.Function(V)

# -- 5a. Displacement Problem --
# Solve for u, using pnew (from previous iteration)
a_u = ufl.inner(epsilon(v), sigma(u, pnew)) * dx
L_u = ufl.inner(fem.Constant(msh, default_scalar_type((0.0, 0.0))), v) * dx

problem_u = fem.petsc.LinearProblem(
    a_u, L_u,
    bcs=bc_u,
    u=unew,
    petsc_options_prefix="disp_",
    petsc_options={"ksp_type": "cg", "pc_type": "hypre"}
)

H_sym = History_Condition(unew, Hold)

E_phi = (
    Gc * l * ufl.inner(ufl.grad(p), ufl.grad(q))
    + ((Gc / l) + 2 * H_sym) * p * q
    - 2 * H_sym * q
) * dx

problem_phi = fem.petsc.LinearProblem(
    ufl.lhs(E_phi), ufl.rhs(E_phi),
    bcs=[],
    u=pnew,
    petsc_options_prefix="phi_",
    petsc_options={"ksp_type": "cg", "pc_type": "hypre"}
)


# -- 5c. History Update Projection --
# Solves Hold_new = Project(max(psi(unew), Hold))
# We define this outside the loop for efficiency
r = ufl.TestFunction(VDG)
h = ufl.TrialFunction(VDG)

# The form to project the NEW history state
proj_lhs = ufl.inner(h, r) * dx
proj_rhs = ufl.inner(History_Condition(unew, Hold), r) * dx

problem_hist = fem.petsc.LinearProblem(
    proj_lhs, proj_rhs, u=Hold, petsc_options_prefix="his_", # We overwrite Hold directly
    petsc_options={"ksp_type": "preonly", "pc_type": "lu"} 
)

import os

outdir = "output"
if MPI.COMM_WORLD.rank == 0:
    os.makedirs(outdir, exist_ok=True)
msh.comm.Barrier()

xdmf = io.XDMFFile(msh.comm, f"{outdir}/results.xdmf", "w")
xdmf.write_mesh(msh)

unew.name = "Displacement"
pnew.name = "PhaseField"


# ----------------------------------------------------
# 6. Time Stepping Loop
# ----------------------------------------------------
if MPI.COMM_WORLD.rank == 0:
    force_file = open("ForcevsDisp.txt", "w")
    print("Starting simulation...")

disp = 0.0
displacement1 = 0.005
total_displacement = 0.007

step1 = 100
step2 = 300

dt1 = displacement1 / step1
dt2 = (total_displacement - displacement1) / step2
print(f"large steps:{dt1:.2e}, small steps:{dt2:.2e}")
counter = 0
tol = 1e-4
while disp < total_displacement:
    if disp < displacement1:
        disp += dt1
    else:
        disp += dt2

    load.value = disp
    counter+=1
    # Initialize iteration variables with previous step results
    u_iter.x.array[:] = unew.x.array
    p_iter.x.array[:] = pnew.x.array
    
    err = 1.0
    it = 0
    max_iter = 500
    
    while err > tol and it < max_iter:
        it += 1
        
        # 1. Solve Displacement (using current pnew)
        problem_u.solve()
        
        # 2. Solve Phase Field (using updated unew and fixed Hold)
        problem_phi.solve()
        
        # 3. Check Convergence (Compare current unew vs previous iteration u_iter)
        diff_u = np.linalg.norm(unew.x.array - u_iter.x.array)
        diff_p = np.linalg.norm(pnew.x.array - p_iter.x.array)
        err = max(diff_u, diff_p)
        
        # Update iteration storage
        u_iter.x.array[:] = unew.x.array
        p_iter.x.array[:] = pnew.x.array

    # ------------------------------------------------
    # Post-Convergence Updates
    # ------------------------------------------------
    
    # 1. Update History Variable (Make the energy increase permanent)
    # Note: problem_hist uses unew (converged) and Hold (old) to compute Hold (new)
    problem_hist.solve() 
    # Write VTK output 
    xdmf.write_function(unew, disp)
    xdmf.write_function(pnew, disp)


    # 2. Calculate Reaction Force
    if int(counter) % 1 == 0:
        # Calculate traction vector on top boundary
        traction = ufl.dot(sigma(unew, pnew), n)
        # Integrate y-component (traction[1]) over boundary 1 (Top)
        Fy = fem.assemble_scalar(fem.form(traction[1] * ds(1)))
        # Gather result for MPI safety
        Fy = msh.comm.allreduce(Fy, op=MPI.SUM)
        
        if MPI.COMM_WORLD.rank == 0:
            print(f"Counter: {counter}, Iteration:{it}")
            print(f"Disp: {disp:.2e}| Fy: {Fy:.2e}")
            force_file.write(f"{disp} {Fy}\n")
            force_file.flush()


if MPI.COMM_WORLD.rank == 0:
    xdmf.close()
    force_file.close()
    print("Simulation completed")

Info    : Reading 'Model2_xr_0.5_yr_0.0_1x1.msh'...
Info    : 529 entities
Info    : 34864 nodes
Info    : 69211 elements
Info    : Done reading 'Model2_xr_0.5_yr_0.0_1x1.msh'
Starting simulation...
large steps:5.00e-05, small steps:6.67e-06
Counter: 1, Iteration:2
Disp: 5.00e-05| Fy: 6.84e+00
Counter: 2, Iteration:2
Disp: 1.00e-04| Fy: 1.37e+01
Counter: 3, Iteration:2
Disp: 1.50e-04| Fy: 2.05e+01
Counter: 4, Iteration:2
Disp: 2.00e-04| Fy: 2.73e+01
Counter: 5, Iteration:2
Disp: 2.50e-04| Fy: 3.42e+01
Counter: 6, Iteration:2
Disp: 3.00e-04| Fy: 4.10e+01
Counter: 7, Iteration:2
Disp: 3.50e-04| Fy: 4.78e+01
Counter: 8, Iteration:2
Disp: 4.00e-04| Fy: 5.47e+01
Counter: 9, Iteration:2
Disp: 4.50e-04| Fy: 6.15e+01
Counter: 10, Iteration:2
Disp: 5.00e-04| Fy: 6.83e+01
Counter: 11, Iteration:2
Disp: 5.50e-04| Fy: 7.51e+01
Counter: 12, Iteration:2
Disp: 6.00e-04| Fy: 8.20e+01
Counter: 13, Iteration:2
Disp: 6.50e-04| Fy: 8.88e+01
Counter: 14, Iteration:2
Disp: 7.00e-04| Fy: 9.56e+01
Counter: 15